In [1]:
using Pkg
Pkg.activate("."); 
Pkg.instantiate()

  Activating environment at `C:\Users\email\OneDrive\Dokumente\Uni\Köln\21WS\ATIS3\GroupProject\ATIES3_group_project-main\Project.toml`


In [2]:
using JuMP, Plots, CPLEX, DataFrames, XLSX, Statistics, Parameters#, IterTools, StatsPlots

# Read in Data

In [3]:
data = Dict()
data["time_series"] = DataFrame(XLSX.readtable("data_ATIS3_new.xlsx", "time_series")...)
data["p_scenarios"] = DataFrame(XLSX.readtable("data_ATIS3_new.xlsx", "s_prices")...)
for h in 1:24
    data["q_scenarios", h] = DataFrame(XLSX.readtable("data_ATIS3_new.xlsx", "s_q_$h")...)
end;

# Define Params

### Sets

In [4]:
# Number of scenarios, devided into FC scenarios and price scenarios
Ω_p = length(data["p_scenarios"].delta_15_da)
Ω_FC = 2*length(data["q_scenarios", 1].neg)
Ω = Ω_FC * Ω_p^2
# Number time periods
T = 5
# Number of hours per day
H = 24
# Number of FC levels (low, mid, high)
L = 3;

### Scalars

In [5]:
# Initial and final period
ti = 745
tf = ti + T-1
# Maximum deviation of DA bid from FC
FC_max_devDA = 0.15
# Risk
β = 0.0
α = 0.95
# Scaling factor FC
q_FC_scal = 0.025
# Scenario probablilities
pi = 1/Ω;

2.5e-5

### Vectors

In [6]:
### Length T
# Central prices in DA
p_DA = data["time_series"][ti:tf,"p_da"]
# 15 and DA prices are not used in the stochastic approach; p_DA * Δp_ instead
#p_15  = data["time_series"][ti:tf,"p_15"]
#p_ID = data["time_series"][ti:tf,"p_id3"]
# Power ForeCasted
q_FC = data["time_series"].fc_wind_bw[ti:tf] .* q_FC_scal
# Forecast specifications; level ϵ [low,mid,high]; hour ϵ [1:24]
FC_level = fill(2, T)
FC_hour = []
FC_day = []
hour = 0
day = 1
for t in 1:T
    hour += 1
    if hour > 24
        hour = 1
        day +=1
    end
    push!(FC_hour, hour)
    push!(FC_day, day)
    if q_FC[t] <= 250*q_FC_scal
        FC_level[t] = 1
    elseif q_FC[t] > 1250*q_FC_scal
        FC_level[t] = 3
    end
end

### Length Ω_p
# Errors 15 and ID prices
Δp_15 = data["p_scenarios"].delta_15_da
Δp_ID = data["p_scenarios"].delta_id3_da

### Length Ω_FC
# FC error sign; 1st stage decission
FC_sgn = vcat(
    fill(1, length(data["q_scenarios", 1].neg)),
    fill(2, length(data["q_scenarios", 1].pos)));

**Explanation FC_type** \
The value of a forecast q_FC, as well as the hour of the day (time lag between FC and realization)
leads to different distributions of the FC error Δq_FC. We distinguish between the levels 1 (low; <250 MW), 2 (mid; 250 MW < Δq_FC < 1250 MW)
and 3 (high; > 1250 MW) for the forecasted value and devide the day in 24h. Depending on the period t, we save the hour and the day. Note that our hour has no connection to the "real" time. Later on, FC_level[t] and FC_hour[t] is used to select the right column of the forecast error distribution matrix Δq_FC. Concrete: Δq_FC[ω_FC, FC_level[t], FC_hour[t]].

### Matrices

In [7]:
# Forecast error matrix
    # Relative error of the forecast used for each scenario ω ϵ Ω,
    # depending on the forecast level F_level being low, mid or high (see text above)
    # and the hour of the day h
Δq_FC = Array{Float64,3}(undef, Ω_FC, L, H)
for h in 1:H
    Δq_FC[:,:,h] = cat(
        cat(data["q_scenarios", h].neg_u250, data["q_scenarios", h].pos_u250, dims=1),
        cat(data["q_scenarios", h].neg, data["q_scenarios", h].pos, dims=1),
        cat(data["q_scenarios", h].neg_o1250, data["q_scenarios", h].pos_o1250, dims=1),
        dims=2)
end
;

# Model as Function

In [8]:
# Read in time dependent Vars
function read_timeDepVars(ti,T)
    tf = ti+T-1
    p_DA = data["time_series"][ti:tf,"p_da"]
    q_FC = data["time_series"].fc_wind_bw[ti:tf] .* q_FC_scal
    FC_level = fill(2, T)
    FC_hour = Int[]
    FC_day = Int[]
    hour = 0
    day = 1
    for t in 1:T
        hour += 1
        if hour > 24
            hour = 1
            day +=1
        end
        push!(FC_hour, hour)
        push!(FC_day, day)
        if q_FC[t] <= 250*q_FC_scal
            FC_level[t] = 1
        elseif q_FC[t] > 1250*q_FC_scal
            FC_level[t] = 3
        end
    end
    return p_DA, q_FC, FC_level, FC_hour
end;

In [25]:
function wind_opt(β=0.0, FC_max_devDA=0.15, ti=300, T=1) # time_cntrl=false, 
    # Data is read in, if time control is selected (to save time, if not needed)
    #if time_cntrl
    p_DA, q_FC, FC_level, FC_hour = read_timeDepVars(ti,T)
    #end
    
    result = DataFrame(
        β = Float64[],
        p_DA = Float64[],
        p_15 = Float64[],
        p_ID = Float64[],
        q_FC = Float64[],
        q_DA = Float64[],
        q_15_neg = Float64[],
        q_15_pos = Float64[],
        R_exp = Float64[],
        R_STD = Float64[],
        CVaR = Float64[],
    )
    
    for t in 1:T
        wind = Model(CPLEX.Optimizer)
        set_silent(wind)

        @variables(wind, begin
            # Powers
            q_DA >= 0
            q_15[1:2]
            # Risk
            η[1:Ω_FC, 1:Ω_p, 1:Ω_p] >= 0
            ζ
        end)    

        q_r = @expression(wind, [ω_FC in 1:Ω_FC], # Realized power
            q_FC[t] * (1 + Δq_FC[ω_FC,FC_level[t],FC_hour[t]]))
        p_15 = @expression(wind, [ω_15 in 1:Ω_p], # Realized 15 price
            p_DA[t] * (1 + Δp_15[ω_15]))
        p_ID = @expression(wind, [ω_ID in 1:Ω_p], # Realized ID price
            p_DA[t] * (1 + Δp_ID[ω_ID]))
        R = @expression(wind, [ω_FC in 1:Ω_FC, ω_15 in 1:Ω_p, ω_ID in 1:Ω_p], # Revenue
            p_DA[t] * q_DA
            + p_15[ω_15] * q_15[FC_sgn[ω_FC]]
            + p_ID[ω_ID] * (q_r[ω_FC] - q_DA - q_15[FC_sgn[ω_FC]]))
        CVaR = @expression(wind, # Risk
            (ζ - 1/(1-α) * sum(sum(sum(
                        pi*η[ω_FC,ω_15,ω_ID]
                    for ω_ID in 1:Ω_p)
                for ω_15 in 1:Ω_p)
            for ω_FC in 1:Ω_FC)));


        @objective(wind, Max,
            sum(sum(sum(
                        pi*R[ω_FC,ω_15,ω_ID] 
                    for ω_ID in 1:Ω_p)
                for ω_15 in 1:Ω_p)
            for ω_FC in 1:Ω_FC)
            + β * CVaR);

        @constraints(wind, begin
            # DA bid within certain percentage of the FC
            MinMaxPowerDA, (1-FC_max_devDA)*q_FC[t] <= q_DA <= (1+FC_max_devDA)*q_FC[t]
            # Limits for total sold power
            MaxPowerTot[ω_FC in 1:Ω_FC], 0 <= q_DA+q_15[FC_sgn[ω_FC]] <= q_r[end]
            # Risk
            CVaRconstr[ω_FC in 1:Ω_FC, ω_15 in 1:Ω_p, ω_ID in 1:Ω_p],
                ζ - R[ω_FC,ω_15,ω_ID] <= η[ω_FC,ω_15,ω_ID]
        end)    

        optimize!(wind)

        push!(result, [
            β,
            p_DA[t],
            Statistics.mean(p_15),
            Statistics.mean(p_ID),
            q_FC[t],
            value.(q_DA),
            value.(q_15[1]),
            value.(q_15[2]),
            Statistics.mean(value.(R[:,:,:])),
            Statistics.std(value.(R[:,:,:])),
            value.(CVaR)
        ])
        IJulia.clear_output(true)
        print(β)
        println(t,"/",T)
    end
    IJulia.clear_output(true)
    return result
end;

24*7=168 \
Mo 2016-02-01 = 745 \
Mo 2016-05-02 = 2928 \
Mo 2016-08-01 = 5112 \
Mo 2016-11-07 = 7465

In [27]:
ti = 2928
T = 24
feb = Dict()
for β in 0:0.3:2
    feb[β] = wind_opt(β, 0.15, ti, T)
end

1/24
2/24
3/24
4/24
5/24
6/24
7/24
8/24
9/24
10/24
11/24
12/24
13/24
14/24
15/24
16/24
17/24
18/24
19/24
20/24
21/24
22/24
23/24
24/24
0.0/2
1/24
2/24
3/24
4/24
5/24
6/24
7/24
8/24
9/24
10/24
11/24
12/24
13/24
14/24
15/24
16/24
17/24
18/24
19/24
20/24
21/24
22/24
23/24
24/24
0.3/2
1/24
2/24
3/24
4/24
5/24
6/24
7/24
8/24
9/24
10/24
11/24
12/24
13/24
14/24
15/24
16/24
17/24
18/24
19/24
20/24
21/24
22/24
23/24
24/24
0.6/2
1/24
2/24
3/24
4/24
5/24
6/24
7/24
8/24
9/24
10/24
11/24
12/24
13/24
14/24
15/24
16/24
17/24
18/24
19/24
20/24
21/24
22/24
23/24
24/24
0.9/2
1/24
2/24
3/24
4/24
5/24
6/24
7/24
8/24
9/24
10/24
11/24
12/24
13/24
14/24
15/24
16/24
17/24
18/24
19/24
20/24
21/24
22/24
23/24
24/24
1.2/2
1/24
2/24
3/24
4/24
5/24
6/24
7/24
8/24
9/24
10/24
11/24
12/24
13/24
14/24
15/24
16/24
17/24
18/24
19/24
20/24
21/24
22/24
23/24
24/24
1.5/2
1/24
2/24
3/24
4/24
5/24
6/24
7/24
8/24
9/24
10/24
11/24
12/24
13/24
14/24
15/24
16/24
17/24
18/24
19/24
20/24
21/24
22/24
23/24
24/24
1.8/2


In [29]:
feb

Dict{Any, Any} with 7 entries:
  0.0 => 24×11 DataFrame…
  1.2 => 24×11 DataFrame…
  0.3 => 24×11 DataFrame…
  1.8 => 24×11 DataFrame…
  0.9 => 24×11 DataFrame…
  1.5 => 24×11 DataFrame…
  0.6 => 24×11 DataFrame…

In [ ]:
feb[0.0]